In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os

In [3]:
data_dir = 'C:\\Users\\Akshay\\Downloads\\Machine Learning Competition\\Hackerearth\\Identify the snake breed'
data_dir

'C:\\Users\\Akshay\\Downloads\\Machine Learning Competition\\Hackerearth\\Identify the snake breed'

In [4]:
test_path = data_dir+'\\Test\\'
train_path = data_dir+'\\Train\\'

In [5]:
train = pd.read_csv(os.path.join(train_path, 'train.csv'))
test = pd.read_csv(os.path.join(test_path, 'test.csv'))
train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [6]:
train.shape

(5508, 2)

In [7]:
test.head()

,image_id
0,7ede553357
1,3500b219e4
2,d43a78d6d9
3,1f36f26994
4,8cb85fc58e


In [8]:
test.shape

(2361, 1)

In [9]:
from tqdm import tqdm
from keras.preprocessing import image

train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['image_id'][i]+'.jpg', target_size=(50,50,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

  0%|          | 0/5508 [00:00<?, ?it/s]C:\Users\Akshay\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 5508/5508 [00:08<00:00, 653.31it/s]


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['breed'] = le.fit_transform(train['breed'])

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5508 entries, 0 to 5507
Data columns (total 2 columns):
image_id    5508 non-null object
breed       5508 non-null int32
dtypes: int32(1), object(1)
memory usage: 64.6+ KB


In [12]:
from keras.utils import to_categorical

y = train['breed'].values
y = to_categorical(y)

In [13]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['image_id'][i]+'.jpg', target_size=(50,50,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)


  0%|          | 0/2361 [00:00<?, ?it/s]C:\Users\Akshay\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 2361/2361 [00:03<00:00, 666.22it/s]


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


In [15]:
from keras.models import Sequential, load_model    
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
import cv2


In [16]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(50,50,1)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(35, activation='softmax'))


In [17]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])


In [18]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Epoch 1/50
138/138 [==============================] - 17s 125ms/step - loss: 3.4400 - accuracy: 0.0883 - val_loss: 3.3955 - val_accuracy: 0.0844
Epoch 2/50
138/138 [==============================] - 20s 147ms/step - loss: 3.3921 - accuracy: 0.0892 - val_loss: 3.3829 - val_accuracy: 0.0844
Epoch 3/50
138/138 [==============================] - 18s 131ms/step - loss: 3.3752 - accuracy: 0.0983 - val_loss: 3.3938 - val_accuracy: 0.0844
Epoch 4/50
138/138 [==============================] - 19s 135ms/step - loss: 3.3320 - accuracy: 0.1026 - val_loss: 3.3669 - val_accuracy: 0.1080
Epoch 5/50
138/138 [==============================] - 18s 127ms/step - loss: 3.2603 - accuracy: 0.1185 - val_loss: 3.3587 - val_accuracy: 0.1198
Epoch 6/50
138/138 [==============================] - 17s 126ms/step - loss: 3.1486 - accuracy: 0.1496 - val_loss: 3.3471 - val_accuracy: 0.1071
Epoch 7/50
138/138 [==============================] - 18s 131ms/step - loss: 2.9991 - accuracy: 0.1784 - val_loss: 3.3980 - val_ac

In [19]:
prediction = model.predict_classes(test)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [20]:
prediction

array([28, 12,  0, ..., 28, 34, 23], dtype=int64)

In [21]:
test1 = pd.read_csv('test.csv')
test1.head()

,image_id
0,7ede553357
1,3500b219e4
2,d43a78d6d9
3,1f36f26994
4,8cb85fc58e


In [22]:
solution = pd.DataFrame({'image_id': test1.image_id, 'breed':prediction})
solution.head()

In [23]:
solution['breed'] = le.inverse_transform(solution['breed'])
solution.head()

,image_id,breed
0,7ede553357,storeria-dekayi
1,3500b219e4,lampropeltis-triangulum
2,d43a78d6d9,agkistrodon-contortrix
3,1f36f26994,thamnophis-sirtalis
4,8cb85fc58e,agkistrodon-contortrix


In [24]:
solution.to_csv('Solution.csv', index = False)